In [1]:
import json
import os
from collections import Counter
from tqdm import tqdm
from collections import defaultdict

import numpy as np
import torch
import yaml
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from transformers import OwlViTProcessor

/ext3/miniconda3/envs/owl_boto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TRAIN_ANNOTATIONS_FILE = "data/ego4d_dummy_train.json"
# VAL_ANNOTATIONS_FILE = "/scratch/hk3820/capstone/data/paco_annotations/paco_ego4d_v1_val.json"
IMAGES_PATH = "/scratch/hk3820/capstone/data/paco_frames/v1/paco_frames"

In [26]:
class OwlDataset(Dataset):
    def __init__(self, processor, annotations_file):
        self.images_dir = IMAGES_PATH
        self.processor = processor
        annotations_file = annotations_file

        with open(annotations_file) as f:
            data = json.load(f)["annotations"]

        self.data = data

    def load_image(self, idx: int) -> Image.Image:
        url = self.data[idx]["image_file_name"]
        path = os.path.join(self.images_dir, os.path.basename(url))
        image = Image.open(path).convert("RGB")
        return image, path

    def load_target(self, idx: int):
        text = [self.data[idx]["pos_queries"] + self.data[idx]["neg_queries"]]
        return text, self.data[idx]["bbox"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, path = self.load_image(idx)
        text, bbox = self.load_target(idx)
        w, h = image.size
        metadata = {
            "width": w,
            "height": h,
            "impath": path,
        }
        
        inputs = self.processor(text=text, images=image, return_tensors="pt")
        inputs['pixel_values'] = inputs['pixel_values'].squeeze(0)
        return inputs, torch.tensor(bbox), metadata


In [27]:
processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")

In [28]:
train_dataset = OwlDataset(processor, TRAIN_ANNOTATIONS_FILE)

In [29]:
inputs, bb, metadata = train_dataset[0]

In [30]:
inputs['pixel_values'].shape

torch.Size([3, 768, 768])

In [31]:
inputs['input_ids'].shape

torch.Size([3, 16])

In [32]:
len(train_dataset)

199

In [10]:
train_dataset = OwlDataset(processor, TRAIN_ANNOTATIONS_FILE)

In [7]:
test_dataset = OwlDataset(processor, TRAIN_ANNOTATIONS_FILE)

In [22]:
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True, num_workers=1)

In [14]:
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True, num_workers=1)

In [27]:
def get_dataloaders():
    processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
    train_dataset = OwlDataset(processor, TRAIN_ANNOTATIONS_FILE)
    test_dataset = OwlDataset(processor, TRAIN_ANNOTATIONS_FILE)
    
    train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True, num_workers=1)
    test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True, num_workers=1)
    return train_dataloader, test_dataloader

In [28]:
train_dataloader, test_dataloader = get_dataloaders()